In [1]:
import os
import csv
from pathlib import Path
import spacy
import en_core_web_sm
import pypandoc
from pypandoc.pandoc_download import download_pandoc
#download_pandoc()


In [2]:
class Skill:
    def __init__(self,name,dir, keyword , groups=None,prerequisites= None):
        filename = name
        filename = filename.replace('/','-')
        filename = filename.replace("\\",'-')
        filename = filename + ".html"
        path = os.path.join(dir, filename)
        path = path.replace("\\",'/')
        self.resource_path = path # for the resource path
        self.keyword_search = keyword  # keyword for searching LLM
        self.group_set = set()
        if groups is not None:    
            self.UpdateGroupSet(groups)
        self.prerequisite  = set()
        if prerequisites is not None:
            self.UpdatePrerequisites(prerequisites)
        

    def UpdateGroupSet(self,groups):
        self.group_set.update(groups)
        #print("skill group set updated.")
      
     
    def UpdatePrerequisites(self,prerequisites):
        self.prerequisite.update(prerequisites)
        #print("prerequisite updated.")
        
class Group:
    def __init__(self,name,skills):
        filename = name
        filename = filename.replace('/','-')
        filename = filename.replace("\\",'-')
        filename = filename + ".html"
        path = os.path.join("group", filename)
        path = path.replace("\\",'/')
        self.resource_path = path # for the resource path
        self.keyword_search = name + " in tech" # keyword for searching LLM
        self.skill_set = skills

    def UpdateSkillSet(self,skill):
        self.skill_set.update(skill)
        #print("group skill set updated.")

    def ChangeKeyword(self,keyword):
        self.keyword_search = keyword

    
    
        

In [3]:
class TechStack:
    def __init__(self):
        self.skill_dict_list = {}
        self.group_dict_list = {}
        self.exact_match_replace_dict_list = {}
        self.partial_match_replace_dict_list = {}
        self.ignore_set = set()
        self.not_found_set = set()
        self.document_pepare_set = set()
        self.nlp = spacy.load('en_core_web_md')
        self.vector_group_dict_list = {}
        self.ImportIgnoreSet()
        self.AllThisWillBeRemoveOnceFinalize()

               
    def AddSkillDictList(self,name,path,keyword,groups=None,prerequisites = None):
        if name not in self.skill_dict_list:
            self.skill_dict_list[name] = Skill(name,path,keyword,groups,prerequisites)
            #print(name,"added in skill_dict_list.")
            if groups is not None:
                for g in groups:
                    if g in self.group_dict_list:
                        self.group_dict_list.get(g).UpdateSkillSet({name})
                        #print(name,"added in",g,".")
                    else:
                        self.group_dict_list[g] = Group(g,{name})
                        #print("new group:",g,"have been created and added",name,".")
        else:
            self.UpdateSkillDictList(name,groups,prerequisites)
                    
    def UpdateSkillDictList(self,name,groups,prerequisites):
        if name in self.skill_dict_list:
            self.skill_dict_list[name].UpdateGroupSet(groups)
            if prerequisites is not None:
                self.skill_dict_list[name].UpdatePrerequisites(prerequisites)
                #print(name,"prerequisites updated.")

    def AddGroupDictList(self,name,skills):
        if skills is not None:
            if name in self.group_dict_list:
                self.UpdateGroupDictList(name,skills)
            else:
                found_set = set()
                for s in skills:
                    if s in self.skill_dict_list:
                        self.skill_dict_list[s].UpdateGroupSet({name})
                        found_set.add(s)  
                        #print(s,"added in",name,"group set.")
                self.group_dict_list[name] = Group(name,found_set)

    def UpdateGroupDictList(self,name,skills):
        if name in self.group_dict_list:
            found_set = set()
            for s in skills:
                if s in self.skill_dict_list:
                      found_set.add(s)  
            self.group_dict_list[name].UpdateSkillSet(found_set)
        else:
            self.AddGroupDictList(name,skills)

    def ImportIgnoreSet(self):
        f = open("ignore.txt", "r")
        for c in f:
            c = c.replace("\n", "")
            self.ignore_set.add(c)
        f.close()

    def ExportSkillDictList(self):
        file_path = "skills.csv"
        with open(file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Name", "Search Keyword","Resource Path","Groups","Prerequisites"])
            for key, value in self.skill_dict_list.items():
                name=key
                search = value.keyword_search
                path = value.resource_path
                groups =""
            
                for g in value.group_set:
                    groups += "["
                    groups += g
                    groups +="]"
                pre = ""
                for p in value.prerequisite:
                    pre += "["
                    pre += p
                    pre +="]"
                writer.writerow([name,search,path,groups,pre])
            file.close()
        with open('skills.txt', 'w') as f:
            for i in self.skill_dict_list:
                f.write(i)
                f.write("\n")
            f.write("amazon web services")
            f.write("\n")
            f.write("azure")
            f.write("\n")
            f.write("google cloud")
            f.write("\n")
            f.close()

    def ExportGroupDictList(self):
        file_path = "groups.csv"
        with open(file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Name", "Search Keyword","Resource Path","skills"])
            for key, value in self.group_dict_list.items():
                name=key
                search = value.keyword_search
                path = value.resource_path
                skills =""
                for s in value.skill_set:
                    skills += "["
                    skills += s
                    skills +="]"
                writer.writerow([name,search,path,skills])
            file.close()



    def ExportMatchReplaceDictList(self):
        file_path = "exact match.csv"
        with open(file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Word", "Replace"])
            for key, value in self.exact_match_replace_dict_list.items():
                writer.writerow([key,value])
            file.close()
        file_path = "partial match.csv"
        with open(file_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Word", "Replace"])
            for key, value in self.partial_match_replace_dict_list.items():
                writer.writerow([key,value])
            file.close()



    def ExportNotFoundSet(self):
        with open('not found.txt', 'w') as f:
            for i in  self.not_found_set:
                f.write(i)
                f.write("\n")
            f.close()

    def Filter(self, text):
        text = text.lower()
        text = text.replace("/"," ")
        if text.find('(') != -1:
            text = text.split("(")[0]
            text = text.rsplit()[0]
            
        words = text.split()
        for word in words:   
            if word in self.ignore_set:
                return str("")
            
        if text in self.exact_match_replace_dict_list:
            text = self.exact_match_replace_dict_list.get(text)
            
        words = text.split()
        new_text = ""
        for word in words:
            if word in self.partial_match_replace_dict_list:
                new_text += self.partial_match_replace_dict_list.get(word) 
                new_text +=" "
            else:
                new_text += word 
                new_text +=" "
        return new_text[:-1]

    def Search(self,text):
        if text in self.skill_dict_list:
            self.document_pepare_set.add(text)
            return True
        if text in self.group_dict_list:
            self.document_pepare_set.add(text)
            return True
        # check for . - space and .js js
        for sdl in  self.skill_dict_list:
            check1 = sdl
            check1 = check1.replace(".","")
            check2 = text
            check2 = check2.replace(".","")
            if check1 == check2:
                self.document_pepare_set.add(sdl)
                return True
            check1 = sdl
            check1 = check1.replace("-"," ")
            check2 = text
            check2 = check2.replace("-"," ")
            if check1 == check2:
                self.document_pepare_set.add(sdl)
                return True
            check1 = sdl
            check1 = check1.replace(" ","")
            check2 = text
            check2 = check2.replace(" ","")
            if check1 == check2:
                self.document_pepare_set.add(sdl)
                return True
            check1 = sdl
            check1 = check1.replace(".js","")
            check1 = check1.replace("js","")
            check2 = text
            check2 = check2.replace(".js","")
            check2 = check2.replace("js","")
            if check1 == check2:
                self.document_pepare_set.add(sdl)
                return True
   
        found = False      
        words = text.split()
        # check word by word
        for word in words:   
            if word in self.skill_dict_list:
                self.document_pepare_set.add(word)
                found = True
            elif word in self.group_dict_list:
                self.document_pepare_set.add(word)
                found = True
        
        return found 
        

    def GenerateLearningResource(self,your_skills, job_skills, company_name):
        skills = set()
        if your_skills is not None:
            skills =  job_skills -  your_skills 
        else:
            skills = job_skills

        if len(skills) == 0:
            print("you are good.")
            return False
        
        self.document_pepare_set.clear()

        for s in skills:
            s = self.Filter(s)
            if s != "":
                found = self.Search(s)
                if found == False:
                    self.not_found_set.add(s)
                    
        if len(self.document_pepare_set) == 0 :
            print("No any learning resource generated.")
            return False
        print(self.document_pepare_set)
        html_content = ""
        for d in self.document_pepare_set:
            path = ""
            if d in self.skill_dict_list:
                v = self.skill_dict_list.get(d)
                path = v.resource_path
            elif d in self.skill_dict_list:
                v = self.group_dict_list.get(d)
                path = v.resource_path
            else:
                continue

            if os.path.isfile(path) == False:
                print(d,"not found in",path)
            else:
                with open(path, "r", encoding="utf-8") as file:
                    title = d.title()
                    html_content +="<h1><u><b>"
                    html_content += title
                    html_content +="</b></u></h1>"
                    html_content += file.read()
                file.close()

        output = pypandoc.convert_text(html_content, 'docx', format='html', outputfile='learning resource.docx')
        if output == "":
            print("Document output sucessfully.")
            return True
        else:
            print("Document output failed")
            return False

    def GroupTextVectorization(self):
        for word in self.group_dict_list:
            if self.nlp.vocab[word].has_vector == True:
                vector_word = self.nlp(word)
                if vector_word not in self.vector_group_dict_list:
                    self.vector_group_dict_list[vector_word] = set()
                self.vector_group_dict_list[vector_word].add(word)

    def VectorSearch(self, word):
        if self.nlp.vocab[word].has_vector == True:
            vector_word = self.nlp(word)
            for vw in self.vector_group_dict_list:
                similarity_score = vector_word.similarity(vw)
                if similarity_score >= 0.9:
                    for w in vector_group_dict_list[vw]:
                        print(w)


    
    def AllThisWillBeRemoveOnceFinalize(self):
        name_list = []
        name_list.append("javascript")
        name_list.append("typescript")
        name_list.append("html")
        name_list.append("css")
        name_list.append("python")
        name_list.append("bash")
        name_list.append("java")
        name_list.append("c++")
        name_list.append("c")
        name_list.append("php")
        name_list.append("rust")
        name_list.append("kotlin")
        name_list.append("ruby")
        name_list.append("lua")
        name_list.append("dart")
        name_list.append("assembly")
        name_list.append("r")
        name_list.append("matlab")
        name_list.append("delphi")
        name_list.append("scala")
        name_list.append("perl")
        name_list.append("elixir")
        name_list.append("objective c")
        name_list.append("haskell")
        name_list.append("gdscript")
        name_list.append("lisp")
        name_list.append("solidity")
        name_list.append("clojure")
        name_list.append("julia")
        name_list.append("erlang")
        name_list.append("fortran")
        name_list.append("prolog")
        name_list.append("zig")
        name_list.append("ada")
        name_list.append("ocaml")
        name_list.append("sas")
        name_list.append("crystal")
        name_list.append("apl")
        name_list.append("flow")
        name_list.append("raku")
        name_list.append("racket")
        name_list.append("xhtml")
        name_list.append("d")
        name_list.append("scratch")
        name_list.append("sql")
        for i in range(len(name_list)):
            keyword = name_list[i] + " programming"
            self.AddSkillDictList(name_list[i], "programming", keyword,{"programming"})
        
        name = "socket"
        self.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming","network"} )
        name = "rpc"
        self.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming","network"} )
        name = "checksum"
        self.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming"} )
        name = "multiprocessing"
        self.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming","concurrency"} )
        name = "multithreading"
        self.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming","concurrency"} )
        name = "asynchronous"
        self.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming","concurrency"} )
        name = "serialization"
        self.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming"} )
        name = "deserialization"
        self.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming"} )
        name = "file io"
        self.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming"} )
        name = "encrypt"
        self.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming","security"} )
        name = "decrypt"
        self.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming","security"} )
        name = "requests"
        self.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming","network","web"} )
        name = "lru cache"
        self.AddSkillDictList(name, "advance programming", name + " programming",{"advance programming"} )
        
        
        # microsoft
        name = "windows"
        self.AddSkillDictList(name, "microsoft", name , {"microsoft","operating systems"} )
        name = "windows server"
        self.AddSkillDictList(name, "microsoft", name , {"microsoft","operating systems","server"} )
        name = "mircosoft sql"
        self.AddSkillDictList(name, "microsoft", "microsoft "+ name , {"microsoft","database","data warehouse","relational"})
        name = "word"
        self.AddSkillDictList(name, "microsoft", "microsoft "+name, {"microsoft","microsoft office"} )
        name = "excel"
        self.AddSkillDictList(name, "microsoft", "microsoft "+name, {"microsoft","microsoft office"} )
        name = "powerpoint"
        self.AddSkillDictList(name, "microsoft", "microsoft "+name, {"microsoft","microsoft office"} )
        name = "visio"
        self.AddSkillDictList(name, "microsoft", "microsoft "+name, {"microsoft","microsoft office"} )
        name = "sharepoint"
        self.AddSkillDictList(name, "microsoft", "microsoft "+name, {"microsoft","microsoft office"} )
        name = "dynamics"
        self.AddSkillDictList(name, "microsoft", "microsoft "+name, {"microsoft","microsoft office"} )
        name = "exchange"
        self.AddSkillDictList(name, "microsoft", "microsoft "+name, {"microsoft","microsoft office"} )
        name = "mircosoft visual studio"
        self.AddSkillDictList(name, "ide", name, {"microsoft","ide"} )
        name = "mircosoft visual code"
        self.AddSkillDictList(name, "ide", name, {"microsoft","ide"} )
        name= "power bi"
        self.AddSkillDictList(name, "microsoft", name ,{"visualization","microsoft"} )
        name = "c#"
        self.AddSkillDictList(name, "programming", name, {"microsoft","programming"} )
        name = "f#"
        self.AddSkillDictList(name, "programming", name, {"microsoft","programming"} )
        name = "powershell"
        self.AddSkillDictList(name, "programming", name, {"microsoft","programming"} )
        name = "visual basic"
        self.AddSkillDictList(name, "programming", name, {"microsoft","programming" })
        name = "visual basic .net"
        self.AddSkillDictList(name, "programming", name, {"microsoft","programming" })
        name = "vba"
        self.AddSkillDictList(name, "programming", name, {"microsoft","programming" })
        name = "asp.mvc"
        self.AddSkillDictList(name, "web", name, {"microsoft","framework" })
        name = "asp.net"
        self.AddSkillDictList(name, "web", name, {"microsoft","framework" })
        name = "asp.net core"
        self.AddSkillDictList(name, "web", name, {"microsoft","framework"} )
        name = ".net"
        self.AddSkillDictList(name, "web",  ".net", {"microsoft","framework" })
        name = ".net core"
        self.AddSkillDictList(name, "web", ".net core", {"microsoft","framework" })
        name = "wpf"
        self.AddSkillDictList(name, "graphical user interface", name, {"microsoft","graphical user interface" })
        name = "linq"
        self.AddSkillDictList(name, "web", name, {"microsoft","framework" })
        name = "directx"
        self.AddSkillDictList(name, "game", name, {"microsoft","framework","game" })
        name = "microsoft game developer kit"
        self.AddSkillDictList(name, "game", name, {"microsoft","framework","game" })
        name = "microsoft quantum"
        self.AddSkillDictList(name, "quantum", name , {"microsoft","quantum"})
        name = "microsoft systems center"
        self.AddSkillDictList(name, "microsoft", name , {"microsoft","network","network montioring"})
        #azure
        name = "cosmodb"
        self.AddSkillDictList(name, "microsoft", "azure " + name, {"microsoft","cloud","azure","database","non-relational"})
        name = "azure sql database"
        self.AddSkillDictList(name, "microsoft",  name, {"microsoft","cloud","azure","database","relational"})
        name = "azure blobs"
        self.AddSkillDictList(name, "microsoft", name, {"microsoft","cloud","azure","storage"})
        name = "azure file"
        self.AddSkillDictList(name, "microsoft", name, {"microsoft","cloud","azure"})
        name = "azure elastic san"
        self.AddSkillDictList(name, "microsoft",  name, {"microsoft","cloud","azure"})
        name = "azure tables"
        self.AddSkillDictList(name, "microsoft",  name, {"microsoft","cloud","azure"})
        name = "azure queues"
        self.AddSkillDictList(name, "microsoft", name, {"microsoft","cloud","azure"})
        name = "azure managed disks"
        self.AddSkillDictList(name, "microsoft",  name, {"microsoft","cloud","azure"})
        name = "azure container storage"
        self.AddSkillDictList(name, "microsoft",  name, {"microsoft","cloud","azure"})
        name = "azure container"
        self.AddSkillDictList(name, "microsoft", name, {"microsoft","cloud","azure","container"})
        name = "azure data lake"
        self.AddSkillDictList(name, "microsoft",  name, {"microsoft","cloud","azure"})
        name = "service bus"
        self.AddSkillDictList(name, "microsoft", "azure " + name, {"microsoft","cloud","azure","message queue"})
        name = "hdinsight"
        self.AddSkillDictList(name, "microsoft", "azure " + name, {"microsoft","cloud","azure","message queue"})
        name = "azure synapse analytics"
        self.AddSkillDictList(name, "microsoft",  name, {"microsoft","cloud","azure","data warehouse","big data"})
        name = "azure ai"
        self.AddSkillDictList(name, "microsoft",  name, {"microsoft","cloud","azure","artificial intelligence"})
        name = "azure pipelines"
        self.AddSkillDictList(name, "microsoft",  name, {"microsoft","cloud","azure","continuous integration and continuous deployment"})
        name = "arm"
        self.AddSkillDictList(name, "microsoft", "azure " + name, {"microsoft","cloud","azure"})
        name = "azure virtual machines"
        self.AddSkillDictList(name, "microsoft", name, {"microsoft","cloud","azure", "virtual machine"})
        name = "azure montior"
        self.AddSkillDictList(name, "microsoft", name,{"microsoft","cloud","azure","reliability","performance monitoring"} )
        name = "azure migration"
        self.AddSkillDictList(name, "microsoft", name,{"microsoft","cloud","azure","reliability","migration"} )
        
        
        #google
        name= "android"
        self.AddSkillDictList(name, "google", name,{"google","operating systems", "mobile"})
        name= "go"
        self.AddSkillDictList(name, "programming", name + " programming",{"google","programming"})
        name= "android studio"
        self.AddSkillDictList(name, "mobile", name,{"google","android", "mobile"})
        name= "android sdk"
        self.AddSkillDictList(name, "mobile", name,{"google","android", "mobile"})
        name= "google docs"
        self.AddSkillDictList(name, "google", name,{"google"})
        name= "google sheets"
        self.AddSkillDictList(name, "google", name,{"google"})
        name= "google slides"
        self.AddSkillDictList(name, "google", name,{"google"})
        name= "google analytics"
        self.AddSkillDictList(name, "google", name,{"google","web"})
        #google cloud
        name = "cloud sql"
        self.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","database","relational"})
        name = "alloydb"
        self.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","database","relational"})
        name = "spanner"
        self.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","database","relational"})
        name = "bigtable"
        self.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","database","big data"})
        name = "firestore"
        self.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","database","document database"})
        name = "firebase realtime database"
        self.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","database","document database"})
        name = "memorystore"
        self.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","database","in memory database"})
        name = "mongodb atlas"
        self.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","database","non-relational"})
        name = "bigquery"
        self.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","data warehouse","big data"})
        name = "dataproc"
        self.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud"})
        name = "google cloud storage"
        self.AddSkillDictList(name, "google", name, {"google","cloud","google cloud","storage"})
        name = "vertex ai"
        self.AddSkillDictList(name, "google", name, {"google","cloud","google cloud","artificial intelligence"})
        name = "cloud tasks"
        self.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","message queue"})
        name = "pub sub"
        self.AddSkillDictList(name, "google", "google " + name, {"google","cloud","google cloud","message queue"})
        name = "google kubernetes engine"
        self.AddSkillDictList(name, "google",  name, {"google","cloud","google cloud","container"})
        name = "google cloud monitoring"
        self.AddSkillDictList(name, "google", name, {"google","cloud","google cloud","performance monitoring"})
        name = "cloud build"
        self.AddSkillDictList(name, "google", "google "+ name, {"google","cloud","google cloud","continuous integration and continuous deployment"})
        name = "google compute engine"
        self.AddSkillDictList(name, "google",  name, {"google","cloud","google cloud","virtual machine"})
        
        #aws
        name = "s3"
        self.AddSkillDictList(name, "amazon", "amazon " + name, {"cloud","amazon web services","storage"})
        name = "fsx"
        self.AddSkillDictList(name, "amazon", "amazon " +name, {"cloud","amazon web services"})
        name = "ebs"
        self.AddSkillDictList(name, "amazon", "amazon " +name, {"cloud","amazon web services","storage"})
        name = "amazon file cache"
        self.AddSkillDictList(name, "amazon", name, {"cloud","amazon web services"})
        name = "drs"
        self.AddSkillDictList(name, "amazon",  "amazon " + name, {"cloud","amazon web services"})
        name = "neptune"
        self.AddSkillDictList(name, "amazon",  "amazon " + name, {"cloud","amazon web services","database","graph database"})
        name = "rds"
        self.AddSkillDictList(name, "amazon", "amazon " + name, {"cloud","amazon web services","database","relational"})
        name = "aurora"
        self.AddSkillDictList(name, "amazon", "amazon " + name, {"cloud","amazon web services","database","relational"})
        name = "dynamodb"
        self.AddSkillDictList(name, "amazon",  "amazon " + name, {"cloud","amazon web services","database","non-relational"})
        name = "redshift"
        self.AddSkillDictList(name, "amazon",  "amazon " + name, {"cloud","amazon web services","data warehouse"})
        name = "elasticache"
        self.AddSkillDictList(name, "amazon",  "amazon " + name, {"cloud","amazon web services","data store"})
        name = "memorydb"
        self.AddSkillDictList(name, "amazon",  "amazon " + name, {"cloud","amazon web services","database","in memory database"})
        name = "documentdb"
        self.AddSkillDictList(name, "amazon",  "amazon " + name, {"cloud","amazon web services","database","document databaset"})
        name = "timestream"
        self.AddSkillDictList(name, "amazon",  "amazon " + name, {"cloud","amazon web services","database","time series"})
        name = "athena"
        self.AddSkillDictList(name, "amazon", "amazon " + name, {"cloud","amazon web services","big data"})
        name = "emr"
        self.AddSkillDictList(name, "amazon",  "amazon " + name, {"cloud","amazon web services","big data"})
        name = "sns"
        self.AddSkillDictList(name, "amazon", "amazon " +  name, {"cloud","amazon web services","message queue","big data"})
        name = "amazon lake formation"
        self.AddSkillDictList(name, "amazon", name, {"cloud","amazon web services"})
        name = "sagemaker"
        self.AddSkillDictList(name, "amazon",  "amazon " + name, {"cloud","amazon web services","artificial intelligence"})
        name = "ecs"
        self.AddSkillDictList(name, "amazon",  "amazon " + name, {"cloud","amazon web services","container"})
        name = "fargate"
        self.AddSkillDictList(name, "amazon",  "amazon " + name, {"cloud","amazon web services","container"})
        name = "amazon lambda"
        self.AddSkillDictList(name, "amazon",  name, {"cloud","amazon web services"})
        name = "amazon vpc"
        self.AddSkillDictList(name, "amazon",  name, {"cloud","amazon web services"})
        name = "cloudwatch"
        self.AddSkillDictList(name, "amazon", "amazon " +  name, {"cloud","amazon web services","performance monitoring"})
        name = "sqs"
        self.AddSkillDictList(name, "amazon",  "amazon " + name, {"cloud","amazon web services","message queue"})
        name = "amazon codepipeline"
        self.AddSkillDictList(name, "amazon",  name, {"cloud","amazon web services","continuous integration and continuous deployment"})
        name = "ec2"
        self.AddSkillDictList(name, "amazon",  "amazon " + name, {"cloud","amazon web services", "virtual machine"})
        
        # apple
        name = "mac"
        self.AddSkillDictList(name, "apple", name + " technology", {"apple","operating systems"} )
        name = "ios"
        self.AddSkillDictList(name, "apple", name + " technology", {"apple","operating systems","mobile"} )
        name = "swift"
        self.AddSkillDictList(name, "programming", name + " programming",{"apple","programming"})
        name = "swiftui"
        self.AddSkillDictList(name, "graphical user interface", name + " graphical user interface",{"apple","graphical user interface"})
        name = "ios sdk"
        self.AddSkillDictList(name, "apple", name ,{"apple","framework", "mobile","ios"})
        name = "xcode"
        self.AddSkillDictList(name, "apple", name ,{"apple","framework"})
        # oracle
        name = "oracle sql"
        self.AddSkillDictList(name, "oracle", name, {"oracle","database"})
        name = "pl sql"
        self.AddSkillDictList(name, "programming", name + " programming", {"oracle","database","programming"})
        name= "adf"
        self.AddSkillDictList(name, "oracle", "oracle "+name + " framework",{"oracle"}  )
        name = "oracle cloud"
        self.AddSkillDictList(name, "oracle", name ,{"oracle","cloud"}  )
        name = "oracle virtual box"
        self.AddSkillDictList(name, "oracle", name ,{"oracle","cloud", "virtual machine"}  )
        name = "oracle c"
        self.AddSkillDictList(name, "oracle", name, {"oracle"})
        name = "ebs"
        self.AddSkillDictList(name, "oracle", "oracle "+name ,{"oracle","cloud"}  )
        name = "oracle monitoring"
        self.AddSkillDictList(name, "oracle", name ,{"oracle","cloud","performance monitoring","reliability"}  )
        name = "soa"
        self.AddSkillDictList(name, "oracle", "oracle "+name ,{"oracle","cloud"}  )
        name = "epm"
        self.AddSkillDictList(name, "oracle", "oracle "+ name ,{"oracle","cloud"}  )
        name = "oracle linux"
        self.AddSkillDictList(name, "oracle", name ,{"oracle", "cloud"} )
        name = "oracle net"
        self.AddSkillDictList(name, "oracle", name ,{"oracle", "cloud"} )
        name = "solaris"
        self.AddSkillDictList(name, "oracle", "oracle "+ name , {"oracle","operating systems"} )
        
        #ibm
        name = "db2"
        self.AddSkillDictList(name, "ibm", "ibm " + name, {"database","ibm"})
        name = "netezza"
        self.AddSkillDictList(name, "ibm", "ibm " + name, {"database","ibm","data warehouse"})
        name = "ibm server"
        self.AddSkillDictList(name, "ibm", name, {"ibm"})
        name = "ibm mq"
        self.AddSkillDictList(name, "ibm", name, {"message queue","ibm"})
        name = "ibm aix"
        self.AddSkillDictList(name, "ibm", name, {"performance monitoring","ibm"})
        name = "ibm watson"
        self.AddSkillDictList(name, "ibm", name + " framework", {"ibm","artificial intelligence"})
        name = "qiskit"
        self.AddSkillDictList(name, "ibm", name , {"ibm","quantum"})
        name = "z-os"
        self.AddSkillDictList(name, "ibm", name , {"ibm","os"})
        name = "informix"
        self.AddSkillDictList(name, "ibm", name , {"ibm","reliability","migration"})
        name = "jcl"
        self.AddSkillDictList(name, "ibm", name + " programming", {"ibm","programming"})
        
        # vmware
        name = "vsphere"
        self.AddSkillDictList(name, "vmware", "vmware " + name, {"cloud","vmware"} )
        name = "vcenter"
        self.AddSkillDictList(name, "vmware",  "vmware " +name, {"cloud","vmware"} )
        name = "esxi"
        self.AddSkillDictList(name, "vmware",  "vmware " +name, {"cloud","vmware"} )
        name = "vcloud"
        self.AddSkillDictList(name, "vmware",  "vmware " +name, {"cloud","vmware"} )
        name = "sdn"
        self.AddSkillDictList(name, "vmware",  "vmware " +name, {"cloud","vmware"} )
        name = "vmware certified professional"
        self.AddSkillDictList(name, "vmware", name, {"vmware"} )
        
        #sap
        name = "hana"
        self.AddSkillDictList(name, "sap", name + " database", {"database","sap"})
        name = "sap bi"
        self.AddSkillDictList(name, "sap", name , {"visualization","sap"})
        name = "sap bw"
        self.AddSkillDictList(name, "sap", name , {"data warehouse","sap"})
        name = "sap basis"
        self.AddSkillDictList(name, "sap", name , {"sap"})
        name = "sap abap"
        self.AddSkillDictList(name, "sap", name , {"sap", "programming"})
        
        # apache
        name = "accumulo"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "activemq"
        self.AddSkillDictList(name, "message queue", "apache " + name , {"apache","message queue"})
        name = "airavata"
        self.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
        name = "airflow"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "allura"
        self.AddSkillDictList(name, "git", "apache " + name , {"apache","git"})
        name = "ambari"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "ant"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "ivy"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "apisix"
        self.AddSkillDictList(name, "microservices", "apache " + name , {"apache","microservices"})
        name = "aries"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "arrow"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "avro"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "axis"
        self.AddSkillDictList(name, "web", "apache " + name , {"apache","web"})
        name = "rampart"
        self.AddSkillDictList(name, "web", "apache " + name , {"apache","web"})
        name = "beam"
        self.AddSkillDictList(name, "big data", "apache " + name , {"apache","big data"})
        name = "bloodhound"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache","git"})
        name = "brooklyn"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "calcite"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "camel"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "carbondata"
        self.AddSkillDictList(name, "big data", "apache " + name , {"apache","big data"})
        name = "cassandra"
        self.AddSkillDictList(name, "database", "apache " + name , {"apache","database","non-relational"})
        name = "cayenne"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "celix"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "cloudstack"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "cacoon"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "commons"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "bcel"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "logging"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "ognl"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "cordova"
        self.AddSkillDictList(name, "mobile", "apache " + name , {"apache"})
        name = "couchdb"
        self.AddSkillDictList(name, "database", "apache " + name , {"apache","database", "non-relational","document database"})
        name = "cxf"
        self.AddSkillDictList(name, "web", "apache " + name , {"apache","web"})
        name = "derby"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "jdo"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "cayenne"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "ldap"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "fortress"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "scimple"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "drill"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "druid"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "empire-db"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "felix"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "flex"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "flink"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "flume"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "fluo"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "freemarker"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "geronimmo"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "groovy"
        self.AddSkillDictList(name, "programming", "apache " + name , {"apache","programming"})
        name = "gump"
        self.AddSkillDictList(name, "continuous integration", "apache " + name , {"apache"})
        name = "hadoop"
        self.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
        name = "hbase"
        self.AddSkillDictList(name, "big data", "apache " + name , {"apache","database","big data"})
        name = "helix"
        self.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
        name = "hive"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache","data warehouse"})
        name = "http server"
        self.AddSkillDictList(name, "web", "apache " + name , {"apache","web"})
        name = "iceberg"
        self.AddSkillDictList(name, "big data", "apache " + name , {"apache","big data"})
        name = "ignite"
        self.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
        name = "impala"
        self.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
        name = "iotdb"
        self.AddSkillDictList(name, "database", "apache " + name , {"apache","database","time series"})
        name = "jackrabbit"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "james"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "jmeter"
        self.AddSkillDictList(name, "testing", "apache " + name , {"apache","testing"})
        name = "kafka"
        self.AddSkillDictList(name, "message queue", "apache " + name , {"apache","message queue"})
        name = "kudu"
        self.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
        name = "kylin"
        self.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
        name = "log4j"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "log4net"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "lucene core"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "solr"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "mahout"
        self.AddSkillDictList(name, "artificial intelligence", "apache " + name , {"apache","machine learning","artificial intelligence"})
        name = "maven"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "doxia"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "messos"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "ftp server"
        self.AddSkillDictList(name, "network", "apache " + name , {"apache","network"})
        name = "mina"
        self.AddSkillDictList(name, "network", "apache " + name , {"apache","network"})
        name = "myface"
        self.AddSkillDictList(name, "web", "apache " + name , {"apache","web"})
        name = "mynewt"
        self.AddSkillDictList(name, "network", "apache " + name , {"apache","network"})
        name = "netbeans"
        self.AddSkillDictList(name, "ide", "apache " + name , {"apache","ide"})
        name = "nifi"
        self.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
        name = "nutch"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache","web crawler"})
        name = "nuttx"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache","os"})
        name = "ofbix"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "oozie"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "openjpa"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "opennlp"
        self.AddSkillDictList(name, "artificial intelligence", "apache " + name , {"apache","natural language processing","artificial intelligence"})
        name = "openoffice"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "orc"
        self.AddSkillDictList(name, "big data", "apache " + name , {"apache","big data"})
        name = "parquet"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "pdfbox"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "mod perl"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "phoenix"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "pig"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "pinot"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "pivot"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "poi"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "xmlbeans"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "pr"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "qpid"
        self.AddSkillDictList(name, "message queue", "apache " + name , {"apache","message queue"})
        name = "rocketmq"
        self.AddSkillDictList(name, "message queue", "apache " + name , {"apache","message queue"})
        name = "roller"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "royale"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "samza"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "shiro"
        self.AddSkillDictList(name, "security", "apache " + name , {"apache","security"})
        name = "singa"
        self.AddSkillDictList(name, "artificial intelligence", "apache " + name , {"apache","artificial intelligence"})
        name = "sling"
        self.AddSkillDictList(name, "web", "apache " + name , {"apache","web"})
        name = "solr"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "spamassassin"
        self.AddSkillDictList(name, "security", "apache " + name , {"apache","security"})
        name = "spark"
        self.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
        name = "storm"
        self.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
        name = "struts"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "subversion"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache","git"})
        name = "superset"
        self.AddSkillDictList(name, "web", "apache " + name , {"apache","web"})
        name = "synapse"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "systemsds"
        self.AddSkillDictList(name, "artificial intelligence", "apache " + name , {"apache","machine learning","artificial intelligence"})
        name = "tapestry"
        self.AddSkillDictList(name, "web", "apache " + name , {"apache","web"})
        name = "tcl"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "tika"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "tinkerpop"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "tomcat"
        self.AddSkillDictList(name, "network", "apache " + name , {"apache","network"})
        name = "tomee"
        self.AddSkillDictList(name, "network", "apache " + name , {"apache","network"})
        name = "traffic server"
        self.AddSkillDictList(name, "network", "apache " + name , {"apache","network"})
        name = "turbine"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "velocity"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "axiom"
        self.AddSkillDictList(name, "web", "apache " + name , {"apache","web"})
        name = "wicket"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "xalan"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "xerces"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "batik"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "fop"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "yetus"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        name = "zookeeper"
        self.AddSkillDictList(name, "distributed", "apache " + name , {"apache","distributed"})
        name = "hdfs"
        self.AddSkillDictList(name, "apache", "apache " + name , {"apache"})
        
        # red hat
        name = "ansible"
        self.AddSkillDictList(name, "red hat", name, {"infrastructure code","red hat"} )
        name = "jboss"
        self.AddSkillDictList(name, "red hat", name, {"red hat","reliability","performance monitoring"} )
        
        # cisco
        
        #database
        name = "mysql"
        self.AddSkillDictList(name, "database", name + " database", {"database","relational"})
        name = "postgresql"
        self.AddSkillDictList(name, "database", name + " database", {"database","relational"})
        name = "mongodb"
        self.AddSkillDictList(name, "database", name + " database", {"database","non-relational","document database"})
        name = "cassandra"
        self.AddSkillDictList(name, "database", name + " database", {"database","non-relational","distributed"})
        name = "redis"
        self.AddSkillDictList(name, "database", name + " database", {"database","non-relational","in memory database"})
        name = "neo4j"
        self.AddSkillDictList(name, "database", name + " database", {"database","non-relational","graph database"})
        name = "sqlite"
        self.AddSkillDictList(name, "database", name + " database", {"database"})
        name = "mariadb"
        self.AddSkillDictList(name, "database", name + " database", {"database"})
        name = "elasticsearch"
        self.AddSkillDictList(name, "database", name + " database", {"database"})
        name = "couchbase"
        self.AddSkillDictList(name, "database", name + " database", {"database"})
        name = "orientdb"
        self.AddSkillDictList(name, "database", name + " database", {"database","non-relational"})
        name = "firebirdsql"
        self.AddSkillDictList(name, "database", name + " database", {"database","relational"})
        name = "influxdb"
        self.AddSkillDictList(name, "database", name + " database", {"database","time series"})
        name = "kdb"
        self.AddSkillDictList(name, "database", name + " database", {"database","time series"})
        name = "prometheus"
        self.AddSkillDictList(name, "database", name + " database", {"database","time series"})
        name = "chroma"
        self.AddSkillDictList(name, "database", name + " database", {"database","vector database"})
        name = "pinecone"
        self.AddSkillDictList(name, "database", name + " database", {"database","vector database"})
        name = "faiss"
        self.AddSkillDictList(name, "database", name + " database", {"database","vector database"})
        name = "tibero"
        self.AddSkillDictList(name, "database", name + " database", {"database"})
        name = "percona xtradb"
        self.AddSkillDictList(name, "database", name + " database", {"database"})
        name = "timescaledb"
        self.AddSkillDictList(name, "database", name + " database", {"database"})
        name = "scylladb"
        self.AddSkillDictList(name, "database", name + " database", {"database"})
        name = "foundationdb"
        self.AddSkillDictList(name, "database", name + " database", {"database"})
        name = "opentsdb"
        self.AddSkillDictList(name, "database", name + " database", {"database"})
        name = "yugabytedb"
        self.AddSkillDictList(name, "database", name + " database", {"database"})
        name = "newsql"
        self.AddSkillDictList(name, "database", name + " database", {"database"})
        #container
        name = "docker"
        self.AddSkillDictList(name, "container", name + " container",{"container"} )
        name = "lxc"
        self.AddSkillDictList(name, "container", name + " container",{"container"} )
        name = "container linux"
        self.AddSkillDictList(name, "container", name ,{"container"} )
        name = "portainer"
        self.AddSkillDictList(name, "container", name + " container",{"container"} )
        name = "messos"
        self.AddSkillDictList(name, "container", name + " container",{"container"} )
        name = "kubernetes"
        self.AddSkillDictList(name, "container", name + " container",{"container"} )
        
        
        # cloud
        name = "suse"
        self.AddSkillDictList(name, "cloud", name + " cloud",{"cloud"} )
        name = "snowflake"
        self.AddSkillDictList(name, "cloud", name, {"cloud","data warehouse"} )
        name = "databricks"
        self.AddSkillDictList(name, "cloud", name, {"cloud","data warehouse"} )
        name = "citrix"
        self.AddSkillDictList(name, "cloud", name, {"cloud"} )
        name = "salesforce"
        self.AddSkillDictList(name, "cloud", name, {"cloud"} )
        name = "servicenow"
        self.AddSkillDictList(name, "cloud", name, {"cloud"} )
        name = "akamai"
        self.AddSkillDictList(name, "cloud", name, {"cloud"} )
        name = "teradata"
        self.AddSkillDictList(name, "cloud", name, {"cloud"} )
        name = "qualys"
        self.AddSkillDictList(name, "security", name, {"cloud","security"} )
        name = "heroku"
        self.AddSkillDictList(name, "cloud", name, {"cloud"} )
        name = "alibaba cloud"
        self.AddSkillDictList(name, "cloud", name, {"cloud"} )
        name = "cloudera"
        self.AddSkillDictList(name, "cloud", name, {"cloud"} )
        name = "firebase"
        self.AddSkillDictList(name, "cloud", name, {"cloud"} )
        
        # update here
        name = "sensu"
        self.AddSkillDictList(name, "cloud", name, {"cloud"} )
        
        name = "talend"
        self.AddSkillDictList(name, "reliability", name, {"migration","reliability","etl"} )
        name = "sharegate"
        self.AddSkillDictList(name, "reliability", name ,{"migration","reliability"} )
        name = "carbonite"
        self.AddSkillDictList(name, "reliability", name ,{"migration","backup","reliability"} )
        name = "fivetran"
        self.AddSkillDictList(name, "reliability", name ,{"migration","reliability"} )
        name = "cloudsfer"
        self.AddSkillDictList(name, "reliability", name ,{"migration","reliability"} )
        name = "appdynamics"
        self.AddSkillDictList(name, "reliability", name ,{"migration","reliability","performance monitoring"} )
        name = "informatica"
        self.AddSkillDictList(name, "reliability", name ,{"migration","reliability"} )
        name = "turbonomic"
        self.AddSkillDictList(name, "reliability", name ,{"migration","reliability"} )
        name = "dynatrace"
        self.AddSkillDictList(name, "reliability", name ,{"migration","reliability","performance monitoring","network monitoring","network"} )
        name = "appdna"
        self.AddSkillDictList(name, "reliability", name ,{"migration","reliability"} )
        name = "laplink"
        self.AddSkillDictList(name, "reliability", name ,{"migration","reliability"} )
        
        # monitoring tool
        name = "datadog"
        self.AddSkillDictList(name, "reliability", name,{"reliability","performance monitoring","network monitoring","network"} )
        name = "new relic"
        self.AddSkillDictList(name, "reliability", name,{"reliability","performance monitoring"} )
        name = "manageengine"
        self.AddSkillDictList(name, "reliability", name,{"reliability","performance monitoring","network monitoring","network"} )
        name = "solarwinds"
        self.AddSkillDictList(name, "security", name,{"reliability","performance monitoring","network monitoring","network","security montioring"} )
        name = "elastic apm"
        self.AddSkillDictList(name, "reliability", name,{"reliability","performance monitoring"} )
        name = "splunk"
        self.AddSkillDictList(name, "reliability", name,{"reliability","performance monitoring"} )
        name = "instana"
        self.AddSkillDictList(name, "reliability", name,{"reliability","performance monitoring"} )
        name = "raygun"
        self.AddSkillDictList(name, "reliability", name + "performance monitoring",{"reliability","performance monitoring"} )
        name = "signoz"
        self.AddSkillDictList(name, "reliability", name,{"reliability","performance monitoring"} )
        name = "eg innovations"
        self.AddSkillDictList(name, "reliability", name,{"reliability","performance monitoring"} )
        
        # pen testing
        name = "nmap"
        self.AddSkillDictList(name, "security", name + " pen test", {"security","penetration testing"})
        name = "w3af"
        self.AddSkillDictList(name, "security", name +" pen test" , {"security","penetration testing"})
        name = "tenable nessus"
        self.AddSkillDictList(name, "security", name + " pen test", {"security","penetration testing"})
        name = "onsecurity"
        self.AddSkillDictList(name, "security", name + " pen test", {"security","penetration testing"})
        name = "wireshark"
        self.AddSkillDictList(name, "security", name + " pen test", {"security","penetration testing","network","network monitoring","security monitoring"})
        name = "zap"
        self.AddSkillDictList(name, "security", name + " pen test", {"security","penetration testing"})
        name = "nikto"
        self.AddSkillDictList(name, "security", name + " pen test", {"security","penetration testing"})
        name = "cobalt"
        self.AddSkillDictList(name, "security", name + " pen test", {"security","penetration testing"})
        name = "rapid7"
        self.AddSkillDictList(name, "security", name + " pen test", {"security","penetration testing"})
        name = "astra"
        self.AddSkillDictList(name, "security", name + " pen test", {"security","penetration testing"})
        
        name = "algosec"
        self.AddSkillDictList(name, "security", name , {"security","security monitoring"})
        name = "security onion"
        self.AddSkillDictList(name, "security", name , {"security","security monitoring"})
        name = "elk stack"
        self.AddSkillDictList(name, "security", name , {"security","security monitoring"})
        name = "cisco stealthwatch"
        self.AddSkillDictList(name, "security", name , {"security","security monitoring"})    
        name = "snort"
        self.AddSkillDictList(name, "security", name , {"security","security monitoring"})    
        name = "suricata"
        self.AddSkillDictList(name, "security", name , {"security","security monitoring"})  
        name = "zeek"
        self.AddSkillDictList(name, "security", name , {"security","security monitoring"})  
        name = "logz.io"
        self.AddSkillDictList(name, "security", name , {"security","security monitoring"}) 
        
        name = "nagios"
        self.AddSkillDictList(name, "network", name,{"network","network monitoring"} )
        name = "prtg"
        self.AddSkillDictList(name, "network", name,{"network","network monitoring"} )
        name = "zabbix"
        self.AddSkillDictList(name, "network", name,{"network","network monitoring"} )
        name = "whatsup gold"
        self.AddSkillDictList(name, "network", name,{"network","network monitoring"} )
        name = "auvik"
        self.AddSkillDictList(name, "network", name,{"network","network monitoring"} )
        name = "icinga"
        self.AddSkillDictList(name, "network", name,{"network","network monitoring"} )
        name = "ninjaone"
        self.AddSkillDictList(name, "network", name,{"network","network monitoring"} )
        name = "cacti"
        self.AddSkillDictList(name, "network", name,{"network","network monitoring"} )
        name = "spiceworks"
        self.AddSkillDictList(name, "network", name,{"network","network monitoring"} )
        name = "checkmk"
        self.AddSkillDictList(name, "network", name,{"network","network monitoring"} )
        name = "kentik"
        self.AddSkillDictList(name, "network", name,{"network","network monitoring"} )
        name = "netcrunch"
        self.AddSkillDictList(name, "network", name,{"network","network monitoring"} )
        name = "observium"
        self.AddSkillDictList(name, "network", name,{"network","network monitoring"} )
        
        name= "whatgraph"
        self.AddSkillDictList(name, "visualization", name ,{"visualization"} )
        name= "grafana"
        self.AddSkillDictList(name, "visualization", name ,{"visualization"} )
        name= "kibana"
        self.AddSkillDictList(name, "visualization", name ,{"visualization"} )
        name= "tableau"
        self.AddSkillDictList(name, "visualization", name ,{"visualization"} )
        name= "infogram"
        self.AddSkillDictList(name, "visualization", name ,{"visualization"} )
        name= "sisense"
        self.AddSkillDictList(name, "visualization", name ,{"visualization"} )
        name= "looker"
        self.AddSkillDictList(name, "visualization", name ,{"visualization"} )
        name= "domo"
        self.AddSkillDictList(name, "visualization", name ,{"visualization"} ) 
        name= "chartblocks"
        self.AddSkillDictList(name, "visualization", name ,{"visualization"} )
        name= "fusioncharts"
        self.AddSkillDictList(name, "visualization", name ,{"visualization"} )
        name= "datawrapper"
        self.AddSkillDictList(name, "visualization", name ,{"visualization"} )
        name= "zoho analytics"
        self.AddSkillDictList(name, "visualization", name ,{"visualization"} )
        name= "qlik"
        self.AddSkillDictList(name, "visualization", name ,{"visualization"} )
        name= "qilkview"
        self.AddSkillDictList(name, "visualization", name ,{"visualization"} )
        name= "qliksense"
        self.AddSkillDictList(name, "visualization", name ,{"visualization"} )
        name= "chart.js"
        self.AddSkillDictList(name, "visualization", name ,{"visualization"} )
        
        
        name = "idrive"
        self.AddSkillDictList(name, "reliability", name ,{"backup","reliability"} )
        name = "veeam"
        self.AddSkillDictList(name, "reliability", name ,{"backup","reliability"} )
        name = "acronis cyber protect"
        self.AddSkillDictList(name, "reliability", name ,{"backup","reliability"} )
        name = "backblaze"
        self.AddSkillDictList(name, "reliability", name ,{"backup","reliability"} )
        name = "easeus"
        self.AddSkillDictList(name, "reliability", name ,{"backup","reliability"} )
        name = "druva"
        self.AddSkillDictList(name, "reliability", name ,{"backup","reliability"} )
        name = "veritas"
        self.AddSkillDictList(name, "reliability", name ,{"backup","reliability"} )
        name = "nakivo"
        self.AddSkillDictList(name, "reliability", name ,{"backup","reliability"} )
        name = "rubrik"
        self.AddSkillDictList(name, "reliability", name ,{"backup","reliability"} )
        name = "r-drive image"
        self.AddSkillDictList(name, "reliability", name ,{"backup","reliability"} )
        name = "retrospect solo"
        self.AddSkillDictList(name, "reliability", name ,{"backup","reliability"} )
        name = "fbackup"
        self.AddSkillDictList(name, "reliability", name ,{"backup","reliability"} )
        name = "shadowprotect"
        self.AddSkillDictList(name, "reliability", name ,{"backup","reliability"} )
        
        name = "hevo data"
        self.AddSkillDictList(name, "etl", name  ,{"etl"})
        name = "stitch data"
        self.AddSkillDictList(name, "etl", name  ,{"etl"})
        name = "fivetran"
        self.AddSkillDictList(name, "etl", name  ,{"etl"})
        
        # end
        
        
        name = "glassfish"
        self.AddSkillDictList(name, "network", name + " application servers",{"network"} )
        name = "jetty"
        self.AddSkillDictList(name, "network", name + " application servers",{"network"} )
        name = "wildfly"
        self.AddSkillDictList(name, "network", name + " application servers",{"network"} )
        
        # top distributed
        name = "mapreduce"
        self.AddSkillDictList(name, "distributed", name + " framework", {"distributed"})
        name = "samza"
        self.AddSkillDictList(name, "distributed", name + " framework", {"distributed"})
        name = "flink"
        self.AddSkillDictList(name, "distributed", name + " framework", {"distributed"})
        name = "heron"
        self.AddSkillDictList(name, "distributed", name + " framework", {"distributed"})
        name = "kudu"
        self.AddSkillDictList(name, "distributed", name + " framework", {"distributed"})
        name = "presto"
        self.AddSkillDictList(name, "distributed", name + " framework", {"distributed"})
        name = "centos"
        self.AddSkillDictList(name, "distributed", name + " framework", {"distributed"})
        name= "pyspark"
        self.AddSkillDictList(name, "distributed", name , {"distributed","framework"},{"python", "spark"}  )
        # mq
        name = "rabbitmq"
        self.AddSkillDictList(name, "message queue", name , {"message queue"})
        name = "ironmq"
        self.AddSkillDictList(name, "message queue", name , {"message queue"})
        name = "mulesoft"
        self.AddSkillDictList(name, "message queue", name + " message queue" , {"message queue"})
        name = "pubsub+"
        self.AddSkillDictList(name, "message queue", name + " message queue" , {"message queue"})
        name = "zeromq"
        self.AddSkillDictList(name, "message queue", name + " message queue" , {"message queue"})
        name = "tibco"
        self.AddSkillDictList(name, "message queue", name + " message queue" , {"message queue"})
        
        # top ai framework
        name = "torch"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","natural language processing"})
        name = "pytorch"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","natural language processing"})
        name = "scikit-learn"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","natural language processing","visualization","data science"})
        name = "tensorflow"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","computer vision","natural language processing"})
        name = "keras"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","computer vision","natural language processing"})
        name = "langchain"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning"})
        name = "cuda"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning"})
        name = "transformers"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning"})
        name = "opennn"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence"})
        name = "openai"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning"})
        name = "pybrain"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence"})
        name = "cntk"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence"})
        name = "dl4j"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence"})
        name = "theano"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence"})
        name = "mxnet"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence"})
        name = "caffe"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence"})
        name = "xgboost"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning"})
        name = "opencv"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","computer vision"})
        name = "viso suite"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","computer vision"})
        name = "yolo"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","computer vision"})
        name = "boofcv"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","computer vision"})
        name = "openvino"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","computer vision"})
        name = "simplecv"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","computer vision"})
        name = "deepface"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","computer vision"})
        name = "spacy"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","natural language processing" })
        name = "xlm"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","natural language processing" })
        name = "chainer"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","natural language processing" })
        name = "gensim"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","natural language processing" })
        name = "nltk"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","natural language processing" })
        name = "matplotlib"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","visualization","data science"})
        name = "seaborn"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","visualization","data science"})
        name = "pandas"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","visualization","data science"})
        name = "numpy"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","visualization","data science"})
        name = "streamlit"
        self.AddSkillDictList(name, "artificial intelligence", name , {"artificial intelligence","data science"})
        name = "gradio"
        self.AddSkillDictList(name, "artificial intelligence", name , {"artificial intelligence","data science"})
        name = "plotly"
        self.AddSkillDictList(name, "artificial intelligence", name , {"artificial intelligence","data science"})
        name = "dash"
        self.AddSkillDictList(name, "artificial intelligence", name , {"artificial intelligence","data science"})
        name = "panel"
        self.AddSkillDictList(name, "artificial intelligence", name , {"artificial intelligence","data science"})
        name = "anvil"
        self.AddSkillDictList(name, "artificial intelligence", name , {"artificial intelligence","data science"})
        name = "data mining"
        self.AddSkillDictList(name, "artificial intelligence", name , {"artificial intelligence","data science"})
        name = "data modeling"
        self.AddSkillDictList(name, "artificial intelligence", name , {"artificial intelligence","data science"})
        name = "language model"
        self.AddSkillDictList(name, "artificial intelligence", name , {"artificial intelligence","data science","natural language processing"})
        name = "cadence"
        self.AddSkillDictList(name, "artificial intelligence", name , {"artificial intelligence","data science"})
        name = "scipy"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","visualization","data science"})
        name = "dataset"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","visualization","data science"})
        name = "dataframe"
        self.AddSkillDictList(name, "artificial intelligence", name + " framework", {"artificial intelligence","machine learning","visualization","data science"})
        # top microservices
        name = "spring"
        self.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"java"})
        name = "spring boot"
        self.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"java"})
        name = "spring cloud"
        self.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"java"})
        name = "expressjs"
        self.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"node.js"})
        name = "django"
        self.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"python"})
        name = "django rest"
        self.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"python"})
        name = "go kit"
        self.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"go"})
        name = "micronaut"
        self.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"java","groovy","kotlin"})
        name = "fastapi"
        self.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"python"})
        name = "helidon"
        self.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"java"})
        name = "lagom"
        self.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"java","scala"})
        name = "quarkus"
        self.AddSkillDictList(name, "microservices", name + " framework", {"microservices"},{"java"})
        
        
        # top game developer
        name = "unreal engine"
        self.AddSkillDictList(name, "game", name + " engine", {"game"},{"c++"})
        name = "unity"
        self.AddSkillDictList(name, "game", name + " engine", {"game"},{"c#"})
        name = "godot"
        self.AddSkillDictList(name, "game", name + " engine", {"game"},{"c#"})
        name = "cocos2d"
        self.AddSkillDictList(name, "game", name + " engine", {"game"})
        name = "rpg maker"
        self.AddSkillDictList(name, "game", name + " engine", {"game"})
        name = "vulkan"
        self.AddSkillDictList(name, "game", name + " graphics framework", {"game"})
        name = "opengl"
        self.AddSkillDictList(name, "game", name + " graphics framework", {"game"})
        name = "physx"
        self.AddSkillDictList(name, "game", name + " physics engine", {"game"})
        name = "bullet"
        self.AddSkillDictList(name, "game", name + " physics engine", {"game"})
        name = "open dynamics engine"
        self.AddSkillDictList(name, "game", name + " physics engine", {"game"})
        name = "fmod"
        self.AddSkillDictList(name, "game", name + " audio framework", {"game"})
        name = "glm"
        self.AddSkillDictList(name, "game", name + " math framework", {"game"})
        name = "assimp"
        self.AddSkillDictList(name, "game", name + " library", {"game"})
        name = "shader"
        self.AddSkillDictList(name, "game", name + " programming", {"game"})
        #quantum
        name = "tensorflow quantum"
        self.AddSkillDictList(name, "quantum", name , {"quantum"})
        name = "cuquantum"
        self.AddSkillDictList(name, "quantum", name , {"quantum"})
        
        
        
        
        
        #top frontend/web
        name = "react"
        self.AddSkillDictList(name, "backend", name + " framework/library", {"frontend","fullstack","web"})
        name = "react.js"
        self.AddSkillDictList(name, "backend", name + " framework/library", {"frontend","fullstack","web"})
        name = "angular"
        self.AddSkillDictList(name, "frontend", name + " framework/library", {"frontend","fullstack","web"})
        name = "angular.js"
        self.AddSkillDictList(name, "frontend", name + " framework/library", {"frontend","fullstack","web"})
        name = "asp.net"
        self.AddSkillDictList(name, "frontend", name + " framework/library", {"frontend","fullstack","web"})
        name = ".net"
        self.AddSkillDictList(name, "frontend", name + " framework/library", {"frontend","fullstack","web"})
        name = "asp.net core"
        self.AddSkillDictList(name, "frontend", name + " framework/library", {"frontend","fullstack","web"})
        name = ".net core"
        self.AddSkillDictList(name, "frontend", name + " framework/library", {"frontend","fullstack","web"})
        name = "vue.js"
        self.AddSkillDictList(name, "frontend", name + " framework/library", {"frontend","fullstack","web"})
        name = "tailwind css"
        self.AddSkillDictList(name, "frontend", name + " framework/library", {"frontend","fullstack"})
        name = "bootstrap"
        self.AddSkillDictList(name, "frontend", name + " framework/library", {"frontend","fullstack"})
        name = "mdl"
        self.AddSkillDictList(name, "frontend", name + " framework/library", {"frontend","fullstack"})
        name = "bulma"
        self.AddSkillDictList(name, "frontend", name + " framework/library", {"frontend","fullstack"})
        name = "semantic ui"
        self.AddSkillDictList(name, "frontend", name + " framework/library", {"frontend","fullstack"})
        name = "foundation"
        self.AddSkillDictList(name, "frontend", name + " framework/library", {"frontend","fullstack"})
        name = "ulkit"
        self.AddSkillDictList(name, "frontend", name + " framework/library", {"frontend","fullstack"})
        name = "node.js"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"})
        name = "nextjs"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"})
        name = "nestjs"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"})
        name = "nuxtjs"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"})
        name = "gatsby"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"})
        name = "flask"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"},{"python"})
        name = "django"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"},{"python"})
        name = "jquery"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"})
        name = "express"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"})
        name = "wordpress"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"})
        name = "spring boot"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"})
        name = "laravel"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"})
        name = "fastapi"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"})
        name = "svelte"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"})
        name = "ruby on rails"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"})
        name = "blazor"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"})
        name = "symfony"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"})
        name = "deno"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"})
        name = "qwik"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"})
        name = "lit"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"})
        name = "play"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"})
        name = "remix"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"})
        name = "solidjs"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"})
        name = "fastify"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"})
        name = "phoenix"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"})
        name = "drupal"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"})
        name = "codelgniter"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"})
        name = "rollup.js"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"})
        name = "babylon.js"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"})
        name = "featherjs"
        self.AddSkillDictList(name, "web", name + " framework/library", {"web"})
        
        # top backend
        name = "spring boot"
        self.AddSkillDictList(name, "backend", name + " backend", {"backend","fullstack"},{"java"})
        name = "django"
        self.AddSkillDictList(name, "backend", name + " backend", {"backend","fullstack"},{"python"})
        name = "expressjs"
        self.AddSkillDictList(name, "backend", name + " backend", {"backend","fullstack"},{"nodejs"})
        name = "laravel"
        self.AddSkillDictList(name, "backend", name + " backend", {"backend","fullstack"},{"php"})
        name = "ruby on rails"
        self.AddSkillDictList(name, "backend", name + " backend", {"backend","fullstack"},{"ruby"})
        name = "asp.net core"
        self.AddSkillDictList(name, "backend", name + " backend", {"backend","fullstack"},{"c#"})
        name = "nestjs"
        self.AddSkillDictList(name, "backend", name + " backend", {"backend","fullstack"},{"typescript","nodejs"})
        name = "koajs"
        self.AddSkillDictList(name, "backend", name + " backend", {"backend","fullstack"},{"nodejs"})
        name = "phoenix"
        self.AddSkillDictList(name, "backend", name + " backend", {"backend","fullstack"},{"elixir"})
        
        
        # top mobile development framework
        name_list.clear()
        name_list.append("flutter")
        name_list.append("react native")
        name_list.append("angular native")
        name_list.append("swiftic")
        name_list.append("kotlin")
        name_list.append("ionic")
        name_list.append("xamarin")
        name_list.append("corona")
        name_list.append("titanium")
        name_list.append("nativescript")
        name_list.append("jquery mobile")
        for i in range(len(name_list)):
            keyword = name_list[i] + " mobile development framework"
            self.AddSkillDictList(name_list[i], "mobile", keyword, {"mobile"})
        
        # gui
        name = "kivy"
        self.AddSkillDictList(name, "graphical user interface", name + " GUI framework", {"graphical user interface"})
        name = "pyqt"
        self.AddSkillDictList(name, "graphical user interface", name + " GUI framework", {"graphical user interface"})
        name = "tkinter"
        self.AddSkillDictList(name, "graphical user interface", name + " GUI framework", {"graphical user interface"})
        name = "pysimplegui"
        self.AddSkillDictList(name, "graphical user interface", name + " GUI framework", {"graphical user interface"})
        name = "pyside2"
        self.AddSkillDictList(name, "graphical user interface", name + " GUI framework", {"graphical user interface"})
        name = "wxpython"
        self.AddSkillDictList(name, "graphical user interface", name + " GUI framework", {"graphical user interface"})
        name = "wax"
        self.AddSkillDictList(name, "graphical user interface", name + " GUI framework", {"graphical user interface"})
        name = "java awt"
        self.AddSkillDictList(name, "graphical user interface", name + " GUI framework", {"graphical user interface"})
        name = "javafx"
        self.AddSkillDictList(name, "graphical user interface", name + " GUI framework", {"graphical user interface"})
        name = "libavg"
        self.AddSkillDictList(name, "graphical user interface", name + " GUI framework", {"graphical user interface"})
        name = "pyforms"
        self.AddSkillDictList(name, "graphical user interface", name + " GUI framework", {"graphical user interface"})
        name = "java swing"
        self.AddSkillDictList(name, "graphical user interface", name + " GUI framework", {"graphical user interface"})
        name = "swt"
        self.AddSkillDictList(name, "graphical user interface", name + " GUI framework", {"graphical user interface"})
        name = "libgdx"
        self.AddSkillDictList(name, "graphical user interface", name + " GUI framework", {"graphical user interface"})
        name = "imgui"
        self.AddSkillDictList(name, "graphical user interface", name + " GUI framework", {"graphical user interface"})
        name = "qt"
        self.AddSkillDictList(name, "graphical user interface", name + " GUI framework", {"graphical user interface"})
        name = "gtk"
        self.AddSkillDictList(name, "graphical user interface", name + " GUI framework", {"graphical user interface"})
        name = "winforms"
        self.AddSkillDictList(name, "graphical user interface", name + " GUI framework", {"graphical user interface"})
        name = "uwp"
        self.AddSkillDictList(name, "graphical user interface", name + " GUI framework", {"graphical user interface"})
        name = "winui"
        self.AddSkillDictList(name, "graphical user interface", name + " GUI framework", {"graphical user interface"})
        name = "xamarin"
        self.AddSkillDictList(name, "graphical user interface", name + " GUI framework", {"graphical user interface"})
        name = ".net maui"
        self.AddSkillDictList(name, "graphical user interface", name + " GUI framework", {"graphical user interface"})
        name = "platform uno"
        self.AddSkillDictList(name, "graphical user interface", name + " GUI framework", {"graphical user interface"})
        name = "avalonia ui"
        self.AddSkillDictList(name, "graphical user interface", name + " GUI framework", {"graphical user interface"})
        name = "flutter"
        self.AddSkillDictList(name, "graphical user interface", name + " GUI framework", {"graphical user interface"})
        name = "material ui"
        self.AddSkillDictList(name, "graphical user interface", name + " GUI framework", {"graphical user interface"})
        name = "uikit"
        self.AddSkillDictList(name, "graphical user interface", name + " GUI framework", {"graphical user interface"})
        name = "chakra ui"
        self.AddSkillDictList(name, "graphical user interface", name + " GUI framework", {"graphical user interface"})
        
        # top test automation frameworks
        name = "selenium"
        self.AddSkillDictList(name, "testing", name + " framework", {"testing"},{"java","python","c#"})
        name = "appium"
        self.AddSkillDictList(name, "testing", name + " framework", {"testing","mobile"})
        name = "testng"
        self.AddSkillDictList(name, "testing", name + " framework", {"testing"},{"java","junit"})
        name = "junit"
        self.AddSkillDictList(name, "testing", name + " framework", {"testing"},{"java"})
        name = "cucumber"
        self.AddSkillDictList(name, "testing", name + " framework", {"testing"})
        name = "testcomplete"
        self.AddSkillDictList(name, "testing", name + " framework", {"testing"},{"javascript","python","visual basic"})
        name = "robot"
        self.AddSkillDictList(name, "testing", name + " framework", {"testing"})
        name = "cypress"
        self.AddSkillDictList(name, "testing", name + " framework", {"testing"},{"javascript"})
        name = "junit jupiter"
        self.AddSkillDictList(name, "testing", name + " framework", {"testing"},{"java","junit"})
        name = "pytest"
        self.AddSkillDictList(name, "testing", name + " framework", {"testing"},{"python"})
        name = "rspec"
        self.AddSkillDictList(name, "testing", name + " framework", {"testing"})
        name = "jenkins"
        self.AddSkillDictList(name, "testing", name + " framework", {"testing","DevOps"})
        name = "galen"
        self.AddSkillDictList(name, "testing", name + " framework", {"testing","web"})
        name = "qtp"
        self.AddSkillDictList(name, "testing", name + " framework", {"testing"})
        name = "uft"
        self.AddSkillDictList(name, "testing", name , {"testing"})
        name = "nunit"
        self.AddSkillDictList(name, "testing", name , {"testing"})
        
        # web crawler
        name = "selenium"
        self.AddSkillDictList(name, "web crawler", name + " framework/library", {"web crawler"})
        name = "selenium base"
        self.AddSkillDictList(name, "web crawler", name + " framework/library", {"web crawler"},{"selenium"})
        name = "scrapy"
        self.AddSkillDictList(name, "web crawler", name + " framework/library", {"web crawler"},{"python"})
        name = "mechanicalsoup"
        self.AddSkillDictList(name, "web crawler", name + " framework/library", {"web crawler"},{"python"})
        name = "beautiful soup"
        self.AddSkillDictList(name, "web crawler", name + " framework/library", {"web crawler"},{"python"})
        name = "pyspider"
        self.AddSkillDictList(name, "web crawler", name + " framework/library", {"web crawler"},{"python"})
        
        
        
        
        # top framework/libraey
        name_list.clear()
        name_list.append("electron")
        name_list.append("lonic")
        name_list.append("cordova")
        name_list.append("tauri")
        name_list.append("capacitor")
        name_list.append("tidyverse")
        name_list.append("quarkus")
        name_list.append("ktor")
        name_list.append("mfc")
        name_list.append("jax")
        name_list.append("jdbc")
        name_list.append("jpa")
        name_list.append("sqlalchemy")
        name_list.append("jsp")
        name_list.append("extjs")
        name_list.append("ejb")
        name_list.append("j2ee")
        name_list.append("durandaljs")
        name_list.append("web3.js")
        name_list.append("casperjs")
        name_list.append("pixijs")
        name_list.append("canvasjs")
        name_list.append("c4.js")
        name_list.append("vanillajs")
        name_list.append("cesiumjs")
        name_list.append("three.js")
        name_list.append("jshint")
        name_list.append("immutable.js")
        name_list.append("knockoutjs")
        name_list.append("polly.js")
        name_list.append("jslint")
        name_list.append("sailsjs")
        name_list.append("sinonjs")
        name_list.append("jsdoc")
        name_list.append("jsf")
        name_list.append("respondjs")
        name_list.append("geojson")
        name_list.append("ado.net")
        name_list.append("ipython")
        name_list.append("j2se")
        name_list.append("jwt")
        name_list.append("boost")
        name_list.append("rxjava")
        name_list.append("jax-rs")
        name_list.append("jee")
        name_list.append("dotnetnuke")
        name_list.append("webpack")
        name_list.append("winform")
        name_list.append("jcr")
        for i in range(len(name_list)):
            keyword = name_list[i] + " framework/library"
            self.AddSkillDictList(name_list[i], "framework", keyword,{"framework"})
        
        # top blockchain
        name = "ethereum"
        self.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
        name = "ibm blockchain"
        self.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
        name = "hyperledger"
        self.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
        name = "eos"
        self.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
        name = "corda"
        self.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
        name = "ripple"
        self.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
        name = "quorum"
        self.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
        name = "iota"
        self.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
        name = "neo"
        self.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
        name = "tezos"
        self.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
        name = "stellar"
        self.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
        name = "sui network"
        self.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
        name = "hedera"
        self.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
        name = "aptos"
        self.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
        name = "flow"
        self.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
        name = "fantom"
        self.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
        name = "solana"
        self.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
        name = "tron"
        self.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
        name = "binance smart chain"
        self.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
        name = "icp"
        self.AddSkillDictList(name, "blockchain", name + " blockchain", {"blockchain"})
        
        name = "fleet"
        self.AddSkillDictList(name, "ide", name ,{"ide"} )
        name = "intellij"
        self.AddSkillDictList(name, "ide", name ,{"ide"} )
        name = "pycharm"
        self.AddSkillDictList(name, "ide", name ,{"ide"} )
        name = "webstorm"
        self.AddSkillDictList(name, "ide", name ,{"ide"} )
        name = "phpstorm"
        self.AddSkillDictList(name, "ide", name ,{"ide"} )
        name = "rider"
        self.AddSkillDictList(name, "ide", name ,{"ide"} )
        name = "clion"
        self.AddSkillDictList(name, "ide", name ,{"ide"} )
        name = "datalore"
        self.AddSkillDictList(name, "ide", name ,{"ide"} )
        name = "datagrip"
        self.AddSkillDictList(name, "ide", name ,{"ide"} )
        name = "rubymine"
        self.AddSkillDictList(name, "ide", name ,{"ide"} )
        name = "goland"
        self.AddSkillDictList(name, "ide", name ,{"ide"} )
        name = "resharper c++"
        self.AddSkillDictList(name, "ide", name ,{"ide"} )
        name = "rustrover"
        self.AddSkillDictList(name, "ide", name ,{"ide"})
        name = "aqua"
        self.AddSkillDictList(name, "ide", name ,{"ide"})
        name = "anaconda"
        self.AddSkillDictList(name, "ide", name ,{"ide"})
        name = "jupyter"
        self.AddSkillDictList(name, "ide", name ,{"ide"})
        name = "eclipse"
        self.AddSkillDictList(name, "ide", name ,{"ide"})
        name = "vert.x"
        self.AddSkillDictList(name, "ide", name ,{"ide"})
        
        name = "gitlab"
        self.AddSkillDictList(name, "git", name ,{"continuous integration and continuous deployment","git","devops"} )
        name = "bitbucket"
        self.AddSkillDictList(name, "git", name ,{"continuous integration and continuous deployment","git","devops"} )
        name = "github"
        self.AddSkillDictList(name, "git", name ,{"git","devops"} )
        name = "preforce"
        self.AddSkillDictList(name, "git", name ,{"git","devops"} )
        
        # cl/cd
        
        self.AddSkillDictList(name, "continuous integration and continuous deployment", name + " CI/CD",{"continuous integration and continuous deployment","devops"} )
        name = "buddy"
        self.AddSkillDictList(name, "continuous integration and continuous deployment", name + " CI/CD",{"continuous integration and continuous deployment","devops"} )
        name = "teamcity"
        self.AddSkillDictList(name, "continuous integration and continuous deployment", name + " CI/CD",{"continuous integration and continuous deployment","devops"} )
        name = "circleci"
        self.AddSkillDictList(name, "continuous integration and continuous deployment", name + " CI/CD",{"continuous integration and continuous deployment","devops"} )
        name = "tracisci"
        self.AddSkillDictList(name, "continuous integration and continuous deployment", name + " CI/CD",{"continuous integration and continuous deployment","devops"} )
        name = "semaphore"
        self.AddSkillDictList(name, "continuous integration and continuous deployment", name + " CI/CD",{"continuous integration and continuous deployment","devops"} )
        name = "puppet"
        self.AddSkillDictList(name, "continuous integration and continuous deployment", name + " CI/CD",{"continuous integration and continuous deployment","devops"} )
        name = "gradle"
        self.AddSkillDictList(name, "continuous integration and continuous deployment", name + " CI/CD",{"continuous integration and continuous deployment","devops"} )
        
        
        
        name= "xml"
        self.AddSkillDictList(name, "format", name + " format",{"format"} )
        name= "json"
        self.AddSkillDictList(name, "format", name + " format",{"format"} )
        name= "binary"
        self.AddSkillDictList(name, "format", name + " format",{"format"} )
        name= "xpath"
        self.AddSkillDictList(name, "format", name + " format",{"format"} )
        name= "query"
        self.AddSkillDictList(name, "format", name + " format",{"format"} )
        name= "ajax"
        self.AddSkillDictList(name, "format", name + " format",{"format"} )
        name= "csv"
        self.AddSkillDictList(name, "format", name + " format",{"format"} )
        name= "hexadecimal"
        self.AddSkillDictList(name, "format", name + " format",{"format"} )
        name= "xsl"
        self.AddSkillDictList(name, "format", name + " format",{"format"} )
        name= "wsdl"
        self.AddSkillDictList(name, "format", name + " format",{"format"} )
        name= "xslt"
        self.AddSkillDictList(name, "format", name + " format",{"format"} )
        name= "xsd"
        self.AddSkillDictList(name, "format", name + " format",{"format"} )
        name= "yaml"
        self.AddSkillDictList(name, "format", name + " format",{"format"} )
        
        # infa code
        name = "terraform"
        self.AddSkillDictList(name, "infrastructure code", name, {"infrastructure code"} )
        name = "spacelift"
        self.AddSkillDictList(name, "infrastructure code", name, {"infrastructure code"} )
        
                              
        name = "ip address"
        self.AddSkillDictList(name, "network", name ,{"network"} )
        name = "mac address"
        self.AddSkillDictList(name, "network", name ,{"network"} )
        name = "firewall"
        self.AddSkillDictList(name, "network", name ,{"network","security"} )
        name = "domain name systems"
        self.AddSkillDictList(name, "network", name ,{"network"} )
        name = "area network"
        self.AddSkillDictList(name, "network", name ,{"network"} )
        name = "storage area network"
        self.AddSkillDictList(name, "network", name ,{"network"} )
        name = "dhcp"
        self.AddSkillDictList(name, "network", name ,{"network"} )
        name = "network protocol"
        self.AddSkillDictList(name, "network", name ,{"network"} )
        name = "network topology"
        self.AddSkillDictList(name, "network", name ,{"network"} )
        name = "network devices"
        self.AddSkillDictList(name, "network", name ,{"network"} )
        name = "broadcasting"
        self.AddSkillDictList(name, "network", name ,{"network"} )
        name = "subnet"
        self.AddSkillDictList(name, "network", name ,{"network"} )
        name = "subnet mask"
        self.AddSkillDictList(name, "network", name ,{"network"} )
        name = "ospf"
        self.AddSkillDictList(name, "network", name ,{"network"} )
        name = "access control"
        self.AddSkillDictList(name, "security", name ,{"network","architecture","security"} )
        name = "xmpp"
        self.AddSkillDictList(name, "network", name ,{"network"} )
        name = "eigrp"
        self.AddSkillDictList(name, "network", name ,{"network"} )
        name = "nfs"
        self.AddSkillDictList(name, "network", name ,{"network"} )
        name = "vrrp"
        self.AddSkillDictList(name, "network", name ,{"network"} )
        name = "mpls"
        self.AddSkillDictList(name, "network", name ,{"network"} )
                              
        
        
        name = "array"
        self.AddSkillDictList(name, "data structure", name + " data structure",{"data structure"} )
        name = "list"
        self.AddSkillDictList(name, "data structure", name + " data structure",{"data structure"} )
        name = "tuple"
        self.AddSkillDictList(name, "data structure", name + " data structure",{"data structure"} )
        name = "stack"
        self.AddSkillDictList(name, "data structure", name + " data structure",{"data structure"} )
        name = "queue"
        self.AddSkillDictList(name, "data structure", name + " data structure",{"data structure"} )
        name = "tree"
        self.AddSkillDictList(name, "data structure", name + " data structure",{"data structure"} )
        name = "graph"
        self.AddSkillDictList(name, "data structure", name + " data structure",{"data structure"} )
        name = "trie"
        self.AddSkillDictList(name, "data structure", name + " data structure",{"data structure"} )
        name = "hash table"
        self.AddSkillDictList(name, "data structure", name + " data structure",{"data structure"} )
        
        name = "abstraction"
        self.AddSkillDictList(name, "object-oriented", name + " oop",{"object-oriented","architecture"} )
        name = "encapsulation"
        self.AddSkillDictList(name, "object-oriented", name + " oop",{"object-oriented","architecture"} )
        name = "inheritance"
        self.AddSkillDictList(name, "object-oriented", name + " oop",{"object-oriented","architecture"} )
        name = "polymorphism"
        self.AddSkillDictList(name, "object-oriented", name + " oop",{"object-oriented","architecture"} )
        name = "object"
        self.AddSkillDictList(name, "object-oriented", name + " oop",{"object-oriented","architecture"} )
        name = "class"
        self.AddSkillDictList(name, "object-oriented", name + " oop",{"object-oriented","architecture"} )
        
        name = "factory"
        self.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
        name = "builder"
        self.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
        name = "dependency injection"
        self.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
        name = "lazy"
        self.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
        name = "object pool"
        self.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
        name = "multition"
        self.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
        name = "raii"
        self.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
        name = "prototype"
        self.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
        name = "singleton"
        self.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
        name = "visitor"
        self.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
        name = "join"
        self.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
        name = "lock"
        self.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
        name = "mvc"
        self.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
        name = "actors"
        self.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
        name = "component"
        self.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
        name = "strategy"
        self.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
        name = "observer"
        self.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
        name = "adapter"
        self.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
        name = "state"
        self.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
        name = "creational"
        self.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
        name = "structural"
        self.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
        name = "behavioral"
        self.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
        name = "decorator"
        self.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
        name = "facade"
        self.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
        name = "mvvm"
        self.AddSkillDictList(name, "design pattern", name + " design pattern",{"design pattern"} )
        
        
        name = "monolithic systems"
        self.AddSkillDictList(name, "systems design", name + " systems design",{"systems design","architecture"} )
        name = "distributed systems"
        self.AddSkillDictList(name, "systems design", name + " systems design",{"systems design","architecture"})
        name = "structured analysis"
        self.AddSkillDictList(name, "systems design", name + " systems design",{"systems design","architecture"})
        name = "structured design"
        self.AddSkillDictList(name, "systems design", name + " systems design",{"systems design","architecture"})
        name = "systems design strategy"
        self.AddSkillDictList(name, "systems design", name + " systems design",{"systems design","architecture"})
        name = "database sharding"
        self.AddSkillDictList(name, "systems design", name + " systems design",{"systems design","architecture"})
        name = "scaling"
        self.AddSkillDictList(name, "systems design", name + " systems design",{"systems design","architecture"})
        name = "caching"
        self.AddSkillDictList(name, "systems design", name + " systems design",{"systems design","architecture"})
        name = "ooad"
        self.AddSkillDictList(name, "systems design", name + " systems design",{"systems design","architecture"})
        name = "public cloud"
        self.AddSkillDictList(name, "systems design", name + " systems design",{"systems design","architecture"})
        name = "private cloud"
        self.AddSkillDictList(name, "systems design", name + " systems design",{"systems design","architecture"})
        name = "hybrid cloud"
        self.AddSkillDictList(name, "systems design", name + " systems design",{"systems design","architecture"})
        name = "file storage"
        self.AddSkillDictList(name, "systems design", name + " systems design",{"systems design","architecture"})
        name = "block storage"
        self.AddSkillDictList(name, "systems design", name + " systems design",{"systems design","architecture"})
        name = "object storage"
        self.AddSkillDictList(name, "systems design", name + " systems design",{"systems design","architecture"})
        name = "availability"
        self.AddSkillDictList(name, "systems design", name + " systems design",{"systems design","architecture"})
        
        #os
        name = "linux"
        self.AddSkillDictList(name, "operating systems", name + " technology", {"operating systems"} )
        name = "vxworks"
        self.AddSkillDictList(name, "operating systems", name + " technology", {"operating systems"} )
        
        name = "soap"
        self.AddSkillDictList(name, "network", name + " networking" ,{"network"} )
        name = "soapui"
        self.AddSkillDictList(name, "testing", name,{"testing","web"} )
        name = "postman"
        self.AddSkillDictList(name, "testing", name,{"testing","web"} )
        
        
        name = "esri"
        self.AddSkillDictList(name, "software or tool", name + " technology",{"arcgis"}  )
        name = "control m"
        self.AddSkillDictList(name, "software or tool", name)
        name = "loadrunner"
        self.AddSkillDictList(name, "software or tool", name)
        name = "tibco ems"
        self.AddSkillDictList(name, "software or tool", name)
        name = "akka"
        self.AddSkillDictList(name, "software or tool", name)
        name = "logstash"
        self.AddSkillDictList(name, "software or tool", name)
        name = "ubuntu"
        self.AddSkillDictList(name, "software or tool", name)
        name = "nginx"
        self.AddSkillDictList(name, "load balancer", name,{"load balancer","architecture"})
        name="ccna"
        self.AddSkillDictList(name, "cisco", name ,{"security","cisco"} )
        name="nessus"
        self.AddSkillDictList(name, "security", name ,{"security"} )
        name="ssl"
        self.AddSkillDictList(name, "security", name ,{"security","network","web"} )
        
        name = "dlp"
        self.AddSkillDictList(name, "security", name ,{"security"} )
        
        
        name= "software development life cycle"
        self.AddSkillDictList(name, "other", name ,{"architecture"})
        name= "agile"
        self.AddSkillDictList(name, "other", name ,{"architecture"})
        name= "gis"
        self.AddSkillDictList(name, "other", name )
        
        self.exact_match_replace_dict_list["aws"]="amazon web services"
        self.exact_match_replace_dict_list["tdd"]="testing"
        self.exact_match_replace_dict_list["webdriver"]="web crawler"
        self.exact_match_replace_dict_list["vbnet"]="visual basic .net"
        self.exact_match_replace_dict_list["vb.net"]="visual basic .net"
        self.exact_match_replace_dict_list["vb"]="visual basic"
        self.exact_match_replace_dict_list["containerization"]="container"
        self.exact_match_replace_dict_list["html5"]="html"
        self.exact_match_replace_dict_list["svn"]="subversion"
        self.exact_match_replace_dict_list["version control"]="git"
        self.exact_match_replace_dict_list["nosql"]="non-relational"
        self.exact_match_replace_dict_list["rdbms"]="relational"
        self.exact_match_replace_dict_list["unity3d"]="unity"
        self.exact_match_replace_dict_list["mssql"]="microsoft sql"
        self.exact_match_replace_dict_list["shaders"]="shader"
        self.exact_match_replace_dict_list["uat"]="testing"
        self.exact_match_replace_dict_list["mui"]="material ui"
        self.exact_match_replace_dict_list["gui"]="graphical user interface"
        self.exact_match_replace_dict_list["ui"]="graphical user interface"
        
        self.partial_match_replace_dict_list["ms"]="microsoft"
        self.partial_match_replace_dict_list["system"]="systems"
        self.partial_match_replace_dict_list["window"]="windows"
        self.partial_match_replace_dict_list["databases"]="database"
        self.partial_match_replace_dict_list["website"]="web"
        self.partial_match_replace_dict_list["test"]="testing"
        self.partial_match_replace_dict_list["networking"]="network"
        self.partial_match_replace_dict_list["solarwinds"]="solarwind"
        
        self.ExportMatchReplaceDictList()
        
        
        f = open("nodeflair skill.txt", "r")
        skills = set()
        for c in f:
            c = c.replace("\n", "")
            if c == '×' or c =='x' or c=="tencent":
                continue
            skills.add(c)
        for s in skills:
            s = self.Filter(s)
            if s != "":
                found = self.Search(s)
                if found == False:
                    self.AddSkillDictList(s, "unknown", s + " in tech" )
        
        
        self.ExportSkillDictList()
        self.ExportGroupDictList()
        self.GroupTextVectorization() 

In [6]:
test = TechStack()
f = open("nodeflair skill.txt", "r")
skills = set()
for c in f:
    c = c.replace("\n", "")
    if c == 'x':
        continue
    skills.add(c)
f.close
result = test.GenerateLearningResource(None, skills, None)
test.ExportNotFoundSet()

{'react native', 'okhttp', 'mesos', 'wicket', 'dojo', 'papertrail', 'karma', 'essbase', 'shiro', 'axway', 'phaser', 'pyspark', 'inversify', 'segment', 'mojolicious', 'codedeploy', 'pentaho', 'jetty', 'json', 'argocd', 'xhtml', 'lightgbm', 'consul', 'swing', 'stata', 'ejb', 'python', 'graylog', 'hal', 'nunit', 'fluentd', 'jdbc', 'c++', 'canvasjs', 'beam', 'rollup.js', 'elasticsearch', 'clearcase', 'vagrant', 'fargate', 'bitrise', 'elasticip', 'postcss', 'ecr', 'logstash', 'websockets', 'openvz', 'appdynamics', 'hooks', 'scss', 'server', 'sonar', 'spring', 'webpack', 'jackson', 'bitbucket', 'yugabytedb', 'tableau', 'prometheus', 'ansible', 'azure data lake', 'oracle', 'mvp', 'telegraf', 'xampp', 'gherkin', 'matlab', 'corda', 'ec2', 'hpux', 'geronimo', 'aws appsync', 'nativescript', 'dapper', 'mockito', 'uikit', 'figma', 'metricbeat', 'talend', 'koin', 'caches', 'gulp', 'unix', 'swagger', 'amcharts', 'graphite', 'glusterfs', 'hd insights', 'mahout', 'scikit-image', 'alerta', 'metal', 'ked

In [7]:
f = open("sample skill.txt", "r")
skills = set()
for c in f:
    c = c.replace("\n", "")
    skills.add(c)
f.close
result = test.GenerateLearningResource(None, skills, None)
self.ExportNotFoundSet()

{'hadoop', 'perl', 'postcss', 'ospf', 'sprint', 'scala', 'vb script', 'mulesoft', 'sap', 'mysql', 'android sdk', 'data mining', 'network', 'systems design', 'sagemaker', 'tibero', 'esri', 'backup', 'visual basic', 'pl sql', '.net', 'android', 'game', 'dlp', 'mq', 'oop', 'rds', 'databricks', 'alteryx', 'kubernetes', 'citrix', 'adobe', 'camel', 'eclipse', 'apache', 'backend', 'json', 'public cloud', 'windows server', 'qtp', 'spark', 'ejb', 'distributed', 'http', 'aurora', 'nessus', 'relational', 'cloudera', 'appdynamics', 'object-oriented', 'lambda', 'airflow', 'ibm', 'dynamics', 'emr', 'soap', 'netezza', 'mybatis', 'sap abap', 'react', 'mobile', 'jboss', 'software development life cycle', 'talend', 'akamai', 'windows', 'ec2', 'programming', 'python', 'numpy', 'metal', 'availability', 'database', 'agile', 'ebs', 'vsphere', 'vrrp', 'hana', 'hybrid cloud', 'arcgis', 'tomcat', 'redshift', 'jmeter', 'iis', 'node.js', 'non-relational', 'stylus', 'teradata', 'epm', 'mariadb', 'dynatrace', 'gro

In [8]:

data_set = ""
f = open("not found.txt", "r")
for c in f:
    c = c.replace("\n", "")
    c= c.lower()
    c = c.replace("/", "-")
    if c.find('(') != -1:
        c = c.split("(")[0]
        c = c.rsplit()[0]
    data_set += c
    data_set += " "
f.close()
split_it = data_set.split() 
counter = {}
for w in split_it:
    if w not in counter:
        counter[w] = 0
    counter[w] +=1
my_keys = sorted(counter, key=counter.get, reverse=True)
print(my_keys)

with open('ignore2.txt', 'w') as f:
    for k in sorted(counter, key=counter.get, reverse=True) :
        f.write(k)
        f.write("\n")
    f.close()





['data', 'systems', 'design', 'analysis', 'technical', 'software', 'it', 'development', 'application', 'engineering', 'technology', 'planning', 'infrastructure', 'project', 'services', 'automation', 'product', 'research', 'deployment', 'computer', 'and', 'integration', 'quality', 'user', 'documentation', 'skills', 'maintenance', 'industry', 'applications', 'process', 'troubleshooting', 'operations', 'tracking', 'modeling', 'desktop', 'information', 'office', 'public', 'tuning', 'payment', 'processing', 'servers', 'solutions', 'knowledge', 'recovery', 'hardware', 'monitoring', 'video', 'visual', 'workflow', 'risk', 'coding', 'production', 'trading', 'service', 'intelligence', 'threat', 'learning', 'reports', 'mathematics', 'disaster', 'encryption', 'plan', 'predictive', 'experience', 'statistical', 'middleware', 'control', 'salesforce.com', 'certified', 'avaya', 'health', 'waterfall', 'improvement', 'operating', 'processes', 'actuarial', 'cyber', 'sdlc', 'graphics', 'automated', 'comput

In [ ]:
#openauth not found in unknown/openauth.html
#elastic bean stalk not found in unknown/elastic bean stalk.html

In [19]:
dict ={}
dir_list = next(os.walk('.'))[1]
for dir in dir_list:
    if dir == ".ipynb_checkpoints" or dir =="downloaded_files" or  dir =="Leetcode" :
        continue
    filenames = next(os.walk(dir), (None, None, []))[2] 
    for f in filenames:
        words = f.rsplit(".")
        extension = words[len(words)-1]
        if extension =="html" and len(words) > 2:
            filename = f.replace(".html","")
            output = pypandoc.convert_file(dir + "/" + f, 'docx', outputfile= dir + "/" + filename +".docx")

print("done")


            



done


In [16]:
import docx
import os

one_word_dict ={}
two_word_dict ={}
three_word_dict ={}
dir_list = next(os.walk('.'))[1]
word_to_ignore =["and","if","let’s","here","use","also","nutshell","into","it’s","be","their","are","its","free","provides","open","source","it","can","be","a","is","to","for","the","of","that","in","an","on","by","as","or","you","from","with","allows","used","your","such","based","like","using","various"]
for dir in dir_list:
    if dir == ".ipynb_checkpoints" or dir =="downloaded_files" or  dir =="Leetcode" or dir=="group":
        continue
    filenames = next(os.walk(dir), (None, None, []))[2]
    for f in filenames:
        words = f.rsplit(".")
        extension = words[len(words)-1]
        if extension =="docx":
            doc = docx.Document(dir + "/" + f)    
            doc_content = "\n".join([paragraph.text for paragraph in doc.paragraphs])
            doc_content = doc_content.lower()
            doc_content = doc_content.replace("\n"," ")
            doc_content = doc_content.replace("certainly","")
            doc_content = doc_content.replace("explore","")
            doc_content = doc_content.replace(",","")
            doc_content = doc_content.replace(".","")
            doc_content = doc_content.replace("/"," ")
            doc_content = doc_content.replace("-"," ")
            doc_content = doc_content.replace("(","")
            doc_content = doc_content.replace(")","")
            doc_content = doc_content.replace("!","")
            doc_content = doc_content.replace("!","")
            doc_content = doc_content.replace("1","")
            doc_content = doc_content.replace("2","")
            doc_content = doc_content.replace("3","")
            #print(doc_content)
            words = doc_content.split()
            for i in range(len(words)):
                one_word = words[i]
                if ':' in one_word or one_word == "here" or one_word == "five":
                    break
                one_word = words[i]
                if one_word in word_to_ignore:
                    continue
                if one_word not in one_word_dict:
                    one_word_dict[one_word] = 0
                one_word_dict[one_word] += 1
                if words[i+1] in word_to_ignore:
                    continue
                two_word = words[i] + " " + words[i+1]
                if two_word not in two_word_dict:
                    two_word_dict[two_word] = 0
                two_word_dict[two_word] += 1
                if words[i+2] in word_to_ignore:
                    continue
                three_word = words[i] + " " + words[i+1] + " " +  words[i+2]
             
                if three_word not in three_word_dict:
                    three_word_dict[three_word] = 0
                
             
                three_word_dict[three_word] += 1
            #print(dir,f)
            #print("\n")


with open('one word keyword.txt', 'w', encoding='utf-8') as f:
    for k in sorted(one_word_dict, key=one_word_dict.get, reverse=True) :
        f.write(k)
        f.write("-")
        f.write(str(one_word_dict[k]))
        f.write("\n")
    f.close()
with open('two word keyword.txt', 'w', encoding='utf-8') as f:
    for k in sorted(two_word_dict, key=two_word_dict.get, reverse=True) :
        f.write(k)
        f.write("-")
        f.write(str(two_word_dict[k]))
        f.write("\n")
    f.close()
with open('three word keyword.txt', 'w', encoding='utf-8') as f:
    for k in sorted(three_word_dict, key=three_word_dict.get, reverse=True) :
        f.write(k)
        f.write("-")
        f.write(str(three_word_dict[k]))
        f.write("\n")
    f.close()


In [15]:
doc = docx.Document("programming/c++.docx")    
doc_content = "\n".join([paragraph.text for paragraph in doc.paragraphs])
doc_content = doc_content.replace(",","")
doc_content = doc_content.replace(".","")
doc_content = doc_content.replace("-"," ")
doc_content = doc_content.replace("/"," ")
doc_content = doc_content.replace(":","")
doc_content = doc_content.replace("Certainly! ","")
doc_content = doc_content.replace("Explore","")
doc_content = doc_content.replace("(","")
doc_content = doc_content.replace(")","")
doc_content = doc_content.replace("!","")
doc_content = doc_content.replace("\n"," ")
doc_content = doc_content.replace("   ","")
doc_content = doc_content.replace("\n"," ")
doc_content = doc_content.replace("   ","")
doc_content = doc_content.lower()
doc_content = doc_content.replace("1","")
doc_content = doc_content.replace("2","")
doc_content = doc_content.replace("3","")
doc_content = doc_content.replace("4","")
doc_content = doc_content.replace("5","")
doc_content = doc_content.replace("6","")
doc_content = doc_content.replace("7","")
doc_content = doc_content.replace("8","")
doc_content = doc_content.replace("9","")
doc_content = doc_content.replace("0","")
print(doc_content)

c++ is a cross platform programming language that extends the capabilities of the c language providing high control over system resources and memory it’s widely used for creating high performance applications operating systems and embedded systems here are five free resources where you can learn c++ wschools c++ introduction this tutorial covers the basics of c++ including syntax variables and development learncppcom a comprehensive website with step by step tutorials examples and quizzes to help you master c++ programming programiz c++ tutorial offers interactive lessons examples and references for learning c++ codecademy c++ course a beginner friendly course that covers c++ essentials for software development official c++ documentation the official documentation provides in depth information about c++ features syntax and libraries you can find it on the c++ standard website happy learning 🚀👩‍💻
